In [1]:
import sys
sys.path.insert(1, './env')
sys.path.insert(1, './agents')

import gym
import pylab
import random
import strategy_imitation, sarsa, ddqn, random_agent, a2c, model_based
import aa_gun
import numpy as np
from collections import deque
import keras
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [ ]:
#массовый тест моделей

In [ ]:
#Проверь на зенитке, на cartpole и на mountain car

score_dict_full={'random':[],'ddqn':[],'a2c':[],'sarsa':[],'mb':[]}
border_med = 100
EPISODES=140
score_dict_med={'random':[],'ddqn':[],'a2c':[],'sarsa':[],'mb':[]}#c border_med по... такты. Надо, чтобы проверить быстроту обучения

agent_list=[random_agent.randomAgent,a2c.A2CAgent,ddqn.DoubleDQNAgent,sarsa.SarsaAgent,model_based.ModelBasedAgent]
for ag_num in [0,3,4]:
    if ag_num==0:
        name='random'
    if ag_num==1:
        name='a2c'
    if ag_num==2:
        name='ddqn' 
    if ag_num==3:
        name='sarsa'
    if ag_num==4:
        name='mb'
    
    for estimation in range(2):
        print('_____',name,estimation,pd.Timestamp.now())
        #здесь весь код от инициализации модели до выдачи scores. Но без рендера.
        # In case of CartPole-v1, maximum length of episode is 500
        
        env = aa_gun.AA_gun_simple0_env()
        #env = gym.make('Seaquest-ramNoFrameskip-v0')
        #env=CartPoleEnv9()
        # get size of state and action from environment
        state_size = env.observation_space.shape[0]
        action_size = env.action_space.n

        #agent = DoubleDQNAgent(state_size, action_size)
        agent = agent_list[ag_num](state_size, action_size)
        agent.train_start=1000
        #agent.train_start=7000
        agent.epsilon_decay=0.9999
        agent.render=False

        scores, episodes = [], []
        reward_lst = []
        s_list=[]
        a_list=[]

        for e in range(EPISODES):
            done = False
            score = 0
            state = env.reset()
            state = np.reshape(state, [1, state_size])

            while not done:
                #if (e in range(2,7)) or (e in range(20,25)) or (e in range(100,103)) or (e in range(200,202)) or (e in range(300,306)) or (e in range(400,406)) or (e in range(500,506)) or (e in range(600,604)):
                #    if agent.render:
                #        env.render()

                # get action for the current state and go one step in environment
                action = agent.get_action(state)
                next_state, reward, done, info = env.step(action)
                next_state = np.reshape(next_state, [1, state_size])
                # if an action make the episode end, then gives penalty of -100

                
                # save the sample <s, a, r, s'> to the replay memory
                agent.append_sample(state, action, reward, next_state, done)
                #if next_state[0,11]!=reward:
                #    print('state[13]!=reward',state[0,11],reward)
                #
                s_list.append(state)
                a_list.append(action)
                reward_lst.append(reward)
                #

                # every time step do the training
                agent.train_model()
                score += reward
                state = next_state

                if done:
                    # every episode update the target model to be same with model
                    agent.update_target_model()

                    # every episode, plot the play time
                    scores.append(score)
                    episodes.append(e)
                    pylab.plot(episodes, scores, 'b')
                    #pylab.savefig("./save_graph/aa_gun_dqn.png")
                    try:
                        print("episode:", e, "  score:", score,np.mean(scores), "  memory length:",
                              len(agent.s), "  epsilon:", agent.epsilon)
                    except Exception:
                        print("episode:", e, "  score:", score,np.mean(scores), "  memory length:",
                              len(agent.memory), "  epsilon:", agent.epsilon)



            # save the model
            #if e % 50 == 0:
            #    agent.model.save_weights("./save_model/aa_gun_dqn.h5")



        #и первые 3000 тактов - это рандом
        #Ходов так 50
        score_dict_full[name].append(np.mean(scores))
        score_dict_med[name].append(np.mean(scores[border_med:]))
        import pickle
        f=open('score_dict.pkl','wb')
        pickle.dump([score_dict_full,score_dict_med],f)
        plt.show()

_____ random 0 2020-02-14 16:55:30.136397
episode: 0   score: 0.375 0.375   memory length: 0   epsilon: 1
hit
episode: 1   score: 1.25 0.8125   memory length: 0   epsilon: 1
hit
episode: 2   score: 1.4375 1.0208333333333333   memory length: 0   epsilon: 1
episode: 3   score: 1.0625 1.03125   memory length: 0   epsilon: 1
episode: 4   score: 1.0 1.025   memory length: 0   epsilon: 1
episode: 5   score: 0.25 0.8958333333333334   memory length: 0   epsilon: 1
hit
episode: 6   score: 2.4375 1.1160714285714286   memory length: 0   epsilon: 1
hit
episode: 7   score: 1.5625 1.171875   memory length: 0   epsilon: 1
hit
hit
episode: 8   score: 2.75 1.3472222222222223   memory length: 0   epsilon: 1
episode: 9   score: 1.4375 1.35625   memory length: 0   epsilon: 1
episode: 10   score: 0.25 1.2556818181818181   memory length: 0   epsilon: 1
hit
episode: 11   score: 2.1875 1.3333333333333333   memory length: 0   epsilon: 1
hit
episode: 12   score: 1.9375 1.3798076923076923   memory length: 0   ep

episode: 109   score: 1.0625 1.3886363636363637   memory length: 0   epsilon: 1
hit
hit
episode: 110   score: 2.25 1.3963963963963963   memory length: 0   epsilon: 1
hit
episode: 111   score: 1.875 1.4006696428571428   memory length: 0   epsilon: 1
episode: 112   score: 1.625 1.4026548672566372   memory length: 0   epsilon: 1
episode: 113   score: 3.8125 1.4237938596491229   memory length: 0   epsilon: 1
hit
hit
episode: 114   score: 3.0625 1.4380434782608695   memory length: 0   epsilon: 1
hit
episode: 115   score: 1.375 1.4375   memory length: 0   epsilon: 1
episode: 116   score: 0.9375 1.4332264957264957   memory length: 0   epsilon: 1
episode: 117   score: 0.75 1.427436440677966   memory length: 0   epsilon: 1
hit
episode: 118   score: 1.5 1.428046218487395   memory length: 0   epsilon: 1
hit
episode: 119   score: 1.9375 1.4322916666666667   memory length: 0   epsilon: 1
hit
episode: 120   score: 1.9375 1.4364669421487604   memory length: 0   epsilon: 1
episode: 121   score: 0.5625

In [ ]:
score_dict_full